In [43]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from urllib.request import Request, urlopen

In [44]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [45]:
def scrape_links(page_number):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://jateng.antaranews.com/daerah/{page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [46]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 6 links from page 1
Total Links: 6


In [47]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"itemprop": "articleBody"})
            # print(body_elem)
            if body_elem:
                # content_elem = body_elem.find_all('br')
                # print(content_elem)
                content_text = body_elem.text
                # for p in content_elem:
                #     content_text += p.text.strip() + "\n"
                
                # if content_text.strip():
                #     content_text=content_text
                # else:
                #     content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jateng',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [48]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()
    

In [49]:
df = pd.DataFrame(results)
print(len(results))
df.head(6)

6


,title,author,category,date,content,region,link
0,Pemkab Batang larang ASN bagikan komentar cap...,Kutnadi\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 10:24 WIB\nRabu, 27 Se...","\n Batang, Jateng (ANTARA) - Pemer...",jateng,https://jateng.antaranews.com/berita/509223/pe...
1,Bank Jateng Cilacap kucurkan Rp969 juta untuk ...,ksm\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 09:58 WIB\nRabu, 27 Se...",\n Semarang (ANTARA) - Bank Jateng...,jateng,https://jateng.antaranews.com/berita/509217/ba...
2,Sentuhan dokter spesialis paliatif tingkatkan ...,Nur Istibaaroh/ksm\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 13:57 WIB\nRabu, 27 Se...",\n Semarang (ANTARA) - Kota Semara...,jateng,https://jateng.antaranews.com/berita/509235/se...
3,Polisi tangkap dua pelaku pencurian mobil di ...,Bambang Dwi Marwoto\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Selasa, 26 September 2023 16:25 WIB\nSelasa, 2...",\n Solo (ANTARA) - Satuan Reskrim ...,jateng,https://jateng.antaranews.com/berita/509238/po...
4,Pertamina sosialisasi pengawasan penyaluran BB...,ksm\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 11:19 WIB\nRabu, 27 Se...",\n Semarang (ANTARA) - Pertamina P...,jateng,https://jateng.antaranews.com/berita/509226/pe...
5,Disdukcapil Kudus targetkan perekaman KTP-el ...,Akhmad Nazaruddin\n\n\t\t\t\t\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 12:32 WIB\nRabu, 27 Se...",\n Kudus (ANTARA) - Dinas Kependud...,jateng,https://jateng.antaranews.com/berita/509229/di...


In [50]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../../tempat_hasil_daerah/jateng_antranews_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')